In [1]:
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup


In [2]:
url = 'http://tiwrmdev.hii.or.th/DATA/REPORT/php/rid_bigcm.html'

res = requests.get(url)
res.encoding = 'utf-8'
print(res)

<Response [200]>


In [3]:
soup = BeautifulSoup(res.content, 'html.parser')

In [4]:
soup.prettify

<bound method Tag.prettify of <html>
<head>
<title>รายงานสถานภาพน้ำเขื่อนต่างๆ วันที่ 27 กันยายน 2564</title>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<style type="text/css"> @import url("https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css"); </style>
<style type="text/css"> @import url("https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css"); </style>
<script src="js/jscalendar-1/calendar.js" type="text/javascript"></script>
<script src="js/jscalendar-1/calendar-setup.js" type="text/javascript"></script>
<script src="js/jscalendar-1/lang/calendar-en.js" type="text/javascript"></script>
<style type="text/css"> @import url('js/jscalendar-1/calendar-win2k-cold-1.css'); </style>
<!-- Piwik Add by atip test web statics 09-nov-2009 -->
<script type="text/javascript">
var pkBaseURL = (("https:" == document.location.protocol) ? "https://www.haii.or.th/piwik/

In [5]:
table = soup.find("table",{"class": "table table-responsive table-hover table-striped table-bordered"})

In [6]:
rows = table.find_all('tbody')[0]

In [7]:
txts = []
for child in rows.children:
    for td in child.select('td[align="right"]'):
        txts.append(td)

nums = []
for txt in txts:
    nums.append(txt.text)

nums.remove('รวมทั้งประเทศ')

In [8]:
dams = pd.read_csv('dams.csv', encoding='utf-8')
dams.head()

,Dam_name,Region,Lat,Lon
0,เขื่อนภูมิพล,เหนือ,17.242500,98.970033
1,เขื่อนสิริกิติ์,เหนือ,17.763889,100.561144
2,เขื่อนแม่งัด,เหนือ,19.159193,98.997210
3,เขื่อนกิ่วลม,เหนือ,18.523598,99.622382
4,เขื่อนแม่กวง,เหนือ,18.926385,99.123319


In [9]:
b=0
m=1
dams.iloc[:,0] = dams.iloc[:,0].str.strip()
names = dams.iloc[:,0].unique()
df = {}

for name in names:
    df[name] = nums[b:b+12]
    b+=12
    m+=1

columns = ['Dam', 'Capacity', '20_vol', 'Curr_vol', 'Curr_vol_pct',
           'Use_vol', 'Use_vol_pct',
           'Yr_avg', 'In_vol', 'In_vol_cum', 'In_vol_yrpct',
           'Out_vol_today', 'Out_vol_cum']

data = pd.DataFrame(df)
data = data.T.reset_index()
data.columns = columns

In [10]:
data = data[data['Dam']!='เขื่อนเจ้าพระยา']
data['Report_date'] = date.today()

In [12]:
data.shape

(41, 14)

In [13]:
his = pd.read_csv('waterreport.csv')
data_c = pd.concat([his,data], axis=0)
data_c.to_csv('waterreport.csv', index=False)